In [1]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.neural_network import MLPClassifier
from imblearn.pipeline import Pipeline
from imblearn.over_sampling import RandomOverSampler
from imblearn.under_sampling import RandomUnderSampler
from sklearn.metrics import roc_auc_score, accuracy_score, f1_score, precision_score, recall_score

In [2]:
import pandas as pd
import numpy as np
import os, psutil
import copy
import matplotlib.pyplot as plt
from ast import literal_eval
from datetime import datetime, timedelta

import pickle
import awswrangler as wr
import boto3
import gc
import math
import pyarrow.parquet as pq
pd.set_option('display.max_columns', None)

from tqdm import tqdm

tqdm.pandas()

In [3]:
df_ref = pd.read_csv('cohort_12-04_selected.csv')
pd.set_option('display.max_columns', None)

/home/ubuntu/.local/lib/python3.6/site-packages/IPython/core/interactiveshell.py:3072: DtypeWarning: Columns (12,21) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [4]:
root = '/mnt/main/new-cohort/'


States = ['State 1', 'State 2', 'State 3', 'State 4', 'State 5', 'State 6', 'State 7']

paths = {x:['', ''] for x in States}

for x in States:
    paths[x][0] = root + x + 'Dataframe_inst.csv'
    paths[x][1] = root + x + 'Dataframe_proc.csv'
    
paths['State 5'][1] = None

## Get POU outcome

In [7]:
#Numpy matrix of NDC codes and metadata for opioid drugs, as determined by the CDC
cdc_opioids = pd.read_csv("~/Desktop/Resources/CDC_Opioids.csv")
cdc_opioids.loc[:,"ABUSE_DETER"] = 0
#Numpy matrix of NDC codes and metadata for opioid drugs with less abuse potential, as determined by the CDC
abuse_deter_opioids = pd.read_csv("~/Desktop/Resources/abuse_deterent.csv")
abuse_deter_opioids.loc[:,"ABUSE_DETER"] = 1
opioid_info = pd.concat([abuse_deter_opioids,cdc_opioids])
opioid_info = opioid_info[opioid_info['Drug'] != 'Buprenorphine']

In [8]:
product_dict = {opioid_info['NDC_Numeric'].loc[i]: opioid_info['Drug'].loc[i] for i in opioid_info.index}
drugs = set(product_dict.values())

product_dict_precise = opioid_info[['NDC_Numeric', 'Drug', 'Strength_Per_Unit', 'MME_Conversion_Factor']].set_index('NDC_Numeric').T.to_dict('list')

/home/ubuntu/.local/lib/python3.6/site-packages/ipykernel_launcher.py:4: UserWarning: DataFrame columns are not unique, some columns will be omitted.
  after removing the cwd from sys.path.


In [9]:
def get_pou_outcome(surgery_dt, claim_from_dt, denied, product_cd, generic_drug_nm, drug_name,
                    drug_list = ['Codeine','Hydrocodone','Hydromorphone','Morphine','Oxycodone','Tramadol'], tolerance = 10):
    
    res = {}

    res['Prolonged_user'] = 0
    
    
    if product_cd != product_cd:
        return res
    
    claim_dates = literal_eval(claim_from_dt)
    codes = literal_eval(product_cd.replace('nan', "'nan'"))
    drug_nm_list = literal_eval(generic_drug_nm.replace('nan', "'nan'"))
    denied_list = literal_eval(denied)
    
    try:
        drug_nm_2_list = literal_eval(drug_name.replace('nan', "'nan'"))
    except:
        drug_nm_2_list = drug_nm_list
        
    surg_dt = datetime.strptime(surgery_dt,'%Y-%m-%d')
    
    
    for i,cd in enumerate(codes):
        
        try:
            cd_ = int(float(cd))
        except:
            cd_ = 0
        
        claim_dt = datetime.strptime(claim_dates[i],'%Y-%m-%d')
        delay = (claim_dt - surg_dt).days
        drug_nm = drug_nm_list[i]
        drug_nm_2 = drug_nm_2_list[i]
        
        
        if cd_ in product_dict:
            
            # Find prolonged opioid user
            if res['Prolonged_user'] == 0 and delay >= 90 and delay <= 180:
                res['Prolonged_user'] = 1
                
        else:
            if res['Prolonged_user'] == 0 and delay >= 90 and delay <= 180:
                for x in drug_list:
                    if x.lower()[:-2] in drug_nm.lower() or x.lower()[:-2] in drug_nm_2.lower():
                        res['Prolonged_user'] = 1
    
    return res

In [10]:
def find_surg_date(c, i):
    if i != i or i[:2] != "20":
        return c
    else:
        return i
    
def select_rx_date(w, c):
    if w != w or len(str(w)) <= 2:
        return c
    else:
        return w

In [ ]:
pou_outcome = pd.DataFrame()
base_columns = ['MA_NUM', 'State', 'CLAIM_FROM_DT', 'ICD9_CD', 'ICD9_DT', 'SURG_DT']
columns_to_load = base_columns + ['EGPCD_RF', 'CLAIM_FROM_DT_rx', 'RX_WRITTEN_DT', 'Denied', 'PRODUCT_CD', 'GENERIC_DRUG_NM', 'DRUG_NAME', 'CLAIM_FROM_DT_inst', 'CLAIM_FROM_DT_dx', 'PLACE_OF_SRVC_CD_dx']

for STATE in States:
    print('Working on {}'.format(STATE))
    temp_MA_nums = set(df_ref[df_ref.State == STATE].MA_NUM.values)
    
    df_1 = pd.read_csv(paths[STATE][0], usecols = columns_to_load)
    df_1['SURG_DT'] = df_1[['CLAIM_FROM_DT','SURG_DT']].apply(lambda x: find_surg_date(x[0], x[1]), axis = 1)
    df_1['Codes_procedures'] = df_1['ICD9_CD']
    
    if not paths[STATE][1]:
        df_2 = pd.DataFrame([], columns = columns_to_load)
    else:
        df_2 = pd.read_csv(paths[STATE][1], usecols = columns_to_load + ['ICD9_CD_proc'])
        df_2['SURG_DT'] = df_2[['CLAIM_FROM_DT','SURG_DT']].apply(lambda x: find_surg_date(x[0], x[1]), axis = 1)
        df_2['Codes_procedures'] = df_2['ICD9_CD_proc']
    
    print('Loaded csvs')
    
    pair_set = set([tuple(x) for x in df_1[['MA_NUM', 'CLAIM_FROM_DT']].values.tolist()])
    df_2 = df_2[df_2[['MA_NUM', 'CLAIM_FROM_DT']].apply(lambda x: tuple(x) not in pair_set , axis = 1)]

    df_1['Source'] = 'Inst'
    df_2['Source'] = 'Proc'

    df = pd.concat([df_1, df_2], axis = 0)[columns_to_load + ['Source','Codes_procedures']].reset_index()
    
    if STATE == 'State 4':
        df = df[df['SURG_DT'].apply(lambda x: x[:4]) != '2020']
    
    
    print('Concatenation done')
    
    
    del df_1
    del df_2
    
    
    if STATE == 'State 5':
        df['RX_DATES'] = df['CLAIM_FROM_DT_rx']
    else:
        df['RX_DATES'] = df[['RX_WRITTEN_DT', 'CLAIM_FROM_DT_rx']].apply(lambda x: select_rx_date(x[0], x[1]), axis = 1)
    print('Creating dataframe')
    
    df = df.sort_values(by='SURG_DT').drop_duplicates(subset = ['MA_NUM'], keep = "last")
    
    df = df[df.MA_NUM.isin(temp_MA_nums)]
    
    df = df[['MA_NUM','SURG_DT', 'RX_DATES', 'Denied', 'PRODUCT_CD', 'GENERIC_DRUG_NM', 'DRUG_NAME']]
    
    temp_pou_outcome = df[['SURG_DT', 'RX_DATES', 'Denied', 'PRODUCT_CD', 'GENERIC_DRUG_NM', 'DRUG_NAME']].progress_apply( lambda x: get_pou_outcome(x[0],
                                                                     x[1],
                                                                     x[2],
                                                                     x[3],
                                                                     x[4],
                                                                     x[5]), 
                                                                     axis = 1) 
    
    temp_pou_outcome = pd.concat([temp_pou_outcome.apply(pd.Series), df[['MA_NUM']]], axis = 1)
    
    pou_outcome = pd.concat([pou_outcome, temp_pou_outcome], axis = 0)

In [13]:
pou_outcome.to_csv('pou_outcome.csv', index = False)

In [5]:
df = pd.read_csv('cohort_12-04_selected.csv')
pou_outcome = pd.read_csv('pou_outcome.csv')
df = df.merge(pou_outcome, on = 'MA_NUM', how = 'left')

In [6]:
columns = set(df.columns) - {"MA_NUM", "HCPCS_CD", "CLAIM_FROM_DT", "ICD9_CD", "ICD9_DT", 
                              "DISCHARG_DT", "POLICY_START_DT", "POLICY_END_DT", "DOB_DT",
                              "LNGCD_RF", "AIDCT_RF", "Source", 'Start_dt', 'End_dt', 'Overdose_Death', "Race", "Outcome_Abuse_3months", "Outcome_Abuse_3months",
                              "Outcome_Abuse_6months", "Outcome_Abuse_12months", "Outcome_Overdose_3months",
                              "Outcome_Overdose_6months", "Outcome_Overdose_12months", "Outcome_Overdose_12months",
                              "Outcome_3months", "Outcome_12months", "Outcome_6months", "ZIP", "ZCTA5", "FIPS", "Main_Provider", 'Continuously_Enrolled_12months'}

columns = [x for x in df.columns if x in columns]

df = df[columns]

columns = [x for x in columns if x!='State']
df =df[columns]

In [7]:
from sklearn.preprocessing import LabelEncoder

label_encoders = {}

for f in ['GENCD_RF', 'Surgery_type']:
    le = LabelEncoder()
    le.fit(df[f])
    label_encoders[f] = le.classes_
    df[f+'_enc'] = le.transform(df[f])
    
df.drop(columns = ['GENCD_RF', 'Surgery_type', 'SURG_DT'], inplace = True)

In [8]:
df.rename(columns = {'Prolonged_user': 'Outcome_6months'}, inplace = True)

# Results with POU outcome

In [9]:
dummies = pd.get_dummies(df['Surgery_type_enc'], prefix='Surgery')
df[dummies.columns] = dummies

In [10]:
new_columns = [x for x in df.columns if (x != 'Surgery_type_enc')]
RF_columns = new_columns
LR_columns = RF_columns

In [11]:
df_train, df_test = train_test_split(df[new_columns], test_size = 0.1, stratify = df['Outcome_6months'], random_state = 1)

X_train, y_train = df_train.drop(columns = ['Outcome_6months']), df_train['Outcome_6months']
X_test, y_test = df_test.drop(columns = ['Outcome_6months']), df_test['Outcome_6months']

X_train = X_train.fillna(X_train.median())
X_test = X_test.fillna(X_test.median())

scaler = StandardScaler()

X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

In [12]:
def get_metrics(y_test, y_pred, thresh = 0.5):
    auc = roc_auc_score(y_test, y_pred)
    f1 = f1_score(y_test, y_pred >= thresh)
    p = precision_score(y_test, y_pred >= thresh)
    r = recall_score(y_test, y_pred >= thresh)
    return auc, f1, p, r
    
def test_split(model, X_test, y_test, random_state = 1, n_splits = 5):
    """Evaluation process: split the test set in 5 splits, evaluate the model on each split and 
       return the average/std of each metric"""
    shuffled = np.concatenate([X_test, y_test.values.reshape(-1,1)], axis = 1)
    
    splits = np.array_split(shuffled, n_splits)
    
    metrics = np.zeros((n_splits, 4))
    
    for i,x in enumerate(splits):
        
        y_pred = model.predict(x[:,:-1])
        try:
            y_pred_split = model.predict_proba(x[:,:-1])[:,1]
        except:
            y_pred_split = model.predict_proba(x[:,:-1])
        y_test_split = x[:,-1]
        metrics[i,:] = get_metrics(y_test_split,y_pred_split, thresh = model.thresh)
    
    res = pd.DataFrame(np.array([metrics.mean(axis = 0), metrics.std(axis = 0)]).T, columns = ['Mean', 'Std'], index = ['AUC', 'F1', 'Precision', 'Recall'])
    return res   

In [13]:
class new_model:
    """ Class used to easily use thresholding for model prediction"""
    def __init__(self, mod, thresh = None):
        self.model = mod
        if thresh:
            self.thresh = thresh    
        return
    
    def predict(self, X_test):
        try:
            proba_predicted = self.model.predict_proba(X_test)[:,1]
        except:
            proba_predicted = self.model.predict_proba(X_test)
        return proba_predicted >= self.thresh
    
    def predict_proba(self, X_test):
        return self.model.predict_proba(X_test)
        
    

## Logistic regression

In [22]:
lr = LogisticRegression()
lr.fit(X_train, y_train)
test_split(new_model(lr, 0.36), X_test, y_test)

,Mean,Std
AUC,0.813840,0.013647
F1,0.573821,0.028539
Precision,0.687318,0.024653
Recall,0.492953,0.031900


## Ridge

In [23]:
ridge = LogisticRegression(penalty = 'l2', C = 0.001)
ridge.fit(X_train, y_train)
test_split(new_model(ridge, 0.32), X_test, y_test)

,Mean,Std
AUC,0.814211,0.013373
F1,0.584788,0.027012
Precision,0.657756,0.024092
Recall,0.526721,0.030646


## Lasso

In [24]:
lasso = LogisticRegression(penalty = 'l1', solver = 'saga', C = 0.05)
lasso.fit(X_train, y_train)
test_split(new_model(lasso, 0.32), X_test, y_test)

/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/_sag.py:329: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did not converge", ConvergenceWarning)


,Mean,Std
AUC,0.814404,0.013333
F1,0.586553,0.025193
Precision,0.657468,0.021446
Recall,0.529773,0.029402


## Elastic net

In [25]:
elnet = LogisticRegression(penalty = 'elasticnet', solver = 'saga', C = 0.01, l1_ratio = 0.5)
elnet.fit(X_train, y_train)
test_split(new_model(elnet, 0.33), X_test, y_test)

/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/_sag.py:329: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did not converge", ConvergenceWarning)


,Mean,Std
AUC,0.814917,0.012928
F1,0.583875,0.026434
Precision,0.670692,0.023875
Recall,0.517349,0.030015


## Random forest

In [26]:
rf = RandomForestClassifier(n_estimators = 300, random_state = 1)
rf.fit(X_train, y_train)
test_split(new_model(rf, 0.35), X_test, y_test)

,Mean,Std
AUC,0.809236,0.011932
F1,0.583497,0.030448
Precision,0.660381,0.025703
Recall,0.523242,0.035854


## XGBoost

In [27]:
import xgboost as xgb

xg = xgb.XGBClassifier(random_state = 1, eta=0.05, reg_lambda = 0.5)
xg.fit(X_train, y_train)
test_split(new_model(xg, 0.35), X_test, y_test)

/home/ubuntu/.local/lib/python3.6/site-packages/xgboost/sklearn.py:1146: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[03:05:44] WARNING: ../src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


/home/ubuntu/.local/lib/python3.6/site-packages/xgboost/data.py:115: UserWarning: Use subset (sliced data) of np.ndarray is not recommended because it will generate extra copies and increase memory consumption
  "memory consumption")
/home/ubuntu/.local/lib/python3.6/site-packages/xgboost/data.py:115: UserWarning: Use subset (sliced data) of np.ndarray is not recommended because it will generate extra copies and increase memory consumption
  "memory consumption")
/home/ubuntu/.local/lib/python3.6/site-packages/xgboost/data.py:115: UserWarning: Use subset (sliced data) of np.ndarray is not recommended because it will generate extra copies and increase memory consumption
  "memory consumption")
/home/ubuntu/.local/lib/python3.6/site-packages/xgboost/data.py:115: UserWarning: Use subset (sliced data) of np.ndarray is not recommended because it will generate extra copies and increase memory consumption
  "memory consumption")
/home/ubuntu/.local/lib/python3.6/site-packages/xgboost/data.py:

,Mean,Std
AUC,0.819188,0.010050
F1,0.588103,0.034049
Precision,0.684794,0.029314
Recall,0.516158,0.039994


## Deep NN

In [28]:
from keras.models import Sequential
from keras.layers import Dense,Dropout,normalization,BatchNormalization,LSTM,GRU
from keras import backend as K
from keras.optimizers import Adam

In [29]:
dnn = Sequential()
dnn.add(Dense(128, activation='relu',input_shape=(X_train.shape[1],)))
dnn.add(Dropout(0.4))
dnn.add(Dense(128, activation='relu',input_shape=(X_train.shape[1],)))
dnn.add(Dropout(0.4))
dnn.add(Dense(128, activation='relu',input_shape=(X_train.shape[1],)))
dnn.add(Dropout(0.4))
dnn.add(Dense(32, activation='relu'))
dnn.add(Dropout(0.4))
dnn.add(Dense(8, activation='relu'))
dnn.add(Dense(1,activation='sigmoid'))

opt = Adam(learning_rate=0.001)
dnn.compile(loss='binary_crossentropy',optimizer=opt,metrics=['AUC'])
history = dnn.fit(X_train,y_train,epochs=10, validation_data = (X_test, y_test))

Epoch 1/10
2728/2728 [==============================] - 7s 2ms/step - loss: 0.4528 - auc: 0.7290 - val_loss: 0.3955 - val_auc: 0.8114
Epoch 2/10
2728/2728 [==============================] - 5s 2ms/step - loss: 0.3870 - auc: 0.8034 - val_loss: 0.3809 - val_auc: 0.8123
Epoch 3/10
2728/2728 [==============================] - 6s 2ms/step - loss: 0.3794 - auc: 0.8118 - val_loss: 0.3868 - val_auc: 0.8129
Epoch 4/10
2728/2728 [==============================] - 5s 2ms/step - loss: 0.3755 - auc: 0.8157 - val_loss: 0.3835 - val_auc: 0.8136
Epoch 5/10
2728/2728 [==============================] - 5s 2ms/step - loss: 0.3746 - auc: 0.8200 - val_loss: 0.3764 - val_auc: 0.8139
Epoch 6/10
2728/2728 [==============================] - 6s 2ms/step - loss: 0.3654 - auc: 0.8257 - val_loss: 0.3770 - val_auc: 0.8158
Epoch 7/10
2728/2728 [==============================] - 5s 2ms/step - loss: 0.3717 - auc: 0.8224 - val_loss: 0.3771 - val_auc: 0.8126
Epoch 8/10
2728/2728 [==============================] - 5s 2ms

In [30]:
test_split(new_model(dnn, 0.31), X_test, y_test)

/home/ubuntu/.local/lib/python3.6/site-packages/tensorflow/python/keras/engine/sequential.py:425: UserWarning: `model.predict_proba()` is deprecated and will be removed after 2021-01-01. Please use `model.predict()` instead.
  warnings.warn('`model.predict_proba()` is deprecated and '
/home/ubuntu/.local/lib/python3.6/site-packages/tensorflow/python/keras/engine/sequential.py:425: UserWarning: `model.predict_proba()` is deprecated and will be removed after 2021-01-01. Please use `model.predict()` instead.
  warnings.warn('`model.predict_proba()` is deprecated and '
/home/ubuntu/.local/lib/python3.6/site-packages/tensorflow/python/keras/engine/sequential.py:425: UserWarning: `model.predict_proba()` is deprecated and will be removed after 2021-01-01. Please use `model.predict()` instead.
  warnings.warn('`model.predict_proba()` is deprecated and '
/home/ubuntu/.local/lib/python3.6/site-packages/tensorflow/python/keras/engine/sequential.py:425: UserWarning: `model.predict_proba()` is depr

,Mean,Std
AUC,0.808575,0.015507
F1,0.579094,0.027629
Precision,0.646955,0.022597
Recall,0.524788,0.034601
